## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
# added for the update of dataframe to find null
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()
city_data_df.describe()

,City_ID,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed
count,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000
mean,346.500000,21.257804,17.396074,69.807767,73.154179,52.708934,8.102176
std,200.484829,33.062686,88.853274,13.845966,19.219793,38.787637,5.784818
min,0.000000,-54.800000,-179.166700,37.060000,1.000000,0.000000,0.000000
25%,173.250000,-4.818100,-62.815625,59.007500,64.000000,11.000000,3.860000
50%,346.500000,26.417100,23.235150,70.980000,77.000000,56.500000,6.610000
75%,519.750000,48.863225,88.339575,79.880000,87.000000,95.000000,11.420000
max,693.000000,78.218600,179.316700,109.060000,100.000000,100.000000,33.310000


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

MaxTemp = int(input("Please enter the maximum temperature you will tolerate. "))
MinTemp = int(input("Please enter the minimum temperature you will tolerate. "))


Please enter the maximum temperature you will tolerate. 85
Please enter the minimum temperature you will tolerate. 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# using the mask technique we learned in class

mask = (city_data_df.MaxTemp <= MaxTemp) & (city_data_df.MaxTemp >= MinTemp)
Temp_df=city_data_df.loc[mask]

In [5]:
# 4a. Determine if there are any empty rows.
#np.where(pd.isnull(city_data_df))
clean_df = Temp_df.replace('', np.nan)
#clean_df.head()
# no empy cells found
clean_df.describe()

,City_ID,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed
count,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000
mean,363.409639,18.048140,38.209782,82.109518,72.481928,56.204819,9.526627
std,198.315900,17.624058,97.063542,1.275186,16.240841,38.928251,6.009283
min,13.000000,-21.200000,-175.200000,80.060000,1.000000,0.000000,0.000000
25%,205.000000,4.866650,-53.201250,80.905000,69.000000,20.000000,4.530000
50%,400.000000,21.116700,70.633300,82.110000,76.000000,71.000000,9.350000
75%,520.500000,32.217050,124.503050,82.810000,83.000000,95.500000,13.800000
max,688.000000,50.350100,172.790200,84.880000,92.000000,100.000000,26.460000


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df.loc[clean_df.isna().any(axis=1)]
#clean_df.head()
clean_df.describe()

,City_ID,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed
count,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000
mean,363.409639,18.048140,38.209782,82.109518,72.481928,56.204819,9.526627
std,198.315900,17.624058,97.063542,1.275186,16.240841,38.928251,6.009283
min,13.000000,-21.200000,-175.200000,80.060000,1.000000,0.000000,0.000000
25%,205.000000,4.866650,-53.201250,80.905000,69.000000,20.000000,4.530000
50%,400.000000,21.116700,70.633300,82.110000,76.000000,71.000000,9.350000
75%,520.500000,32.217050,124.503050,82.810000,83.000000,95.500000,13.800000
max,688.000000,50.350100,172.790200,84.880000,92.000000,100.000000,26.460000


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "MaxTemp", "Current Description", "Lat", "Lng"]].copy().reset_index(drop=True)

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_names = []
hotel_df.head(10)


,City,Country,MaxTemp,Current Description,Lat,Lng,Hotel Name
0,Thinadhoo,MV,80.06,broken clouds,0.5333,72.9333,
1,Sompeta,IN,82.04,overcast clouds,18.9333,84.6000,
2,Hithadhoo,MV,80.08,broken clouds,-0.6000,73.0833,
3,Katsuura,JP,82.22,overcast clouds,35.1333,140.3000,
4,Nanton,CA,80.33,few clouds,50.3501,-113.7687,
5,Butaritari,KI,82.13,overcast clouds,3.0707,172.7902,
6,Kieta,PG,82.49,light rain,-6.2167,155.6333,
7,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,
8,Rapar,IN,81.82,broken clouds,23.5667,70.6333,
9,Merta,IN,84.06,overcast clouds,26.6500,74.0333,


In [8]:
#6a Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key" : g_key
}

# 6b. Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # 6c.  Get the latitude and longitude from the DataFrame.
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
        
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
names=hotel_df[ hotel_df['Hotel Name'] ==""].index
hotel_df.drop(names, inplace = True)
hotel_df.head()

,City,Country,MaxTemp,Current Description,Lat,Lng,Hotel Name
1,Sompeta,IN,82.04,overcast clouds,18.9333,84.6000,బెంకిలి భైరొడమ్మ క్రికెట్ గ్రౌండ్
2,Hithadhoo,MV,80.08,broken clouds,-0.6000,73.0833,Scoop Guest House
3,Katsuura,JP,82.22,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
4,Nanton,CA,80.33,few clouds,50.3501,-113.7687,Changing Seasons Bed and Breakfast
5,Butaritari,KI,82.13,overcast clouds,3.0707,172.7902,Isles Sunset Lodge


In [10]:
# 8a. Create the output File (CSV)
clean_hotel_df = pd.DataFrame(hotel_df)
#clean_hotel_df.head(10)
# 8b. Export the City_Data into a csv
output_data_file = "WeatherPy_vacation.csv"
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>City</dt>{City}<dd>
<dt>Country</dt>{Country}<dd>
<dt>Weather Description</dt>{Current Description}<dd>
<dt>Maximum Temperature</dt>{MaxTemp}<dd>
</d1>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]



In [15]:
# 11a. Add a marker layer for each city to the map.

fig=gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))